# The Laplacian operator

In this practical work, we study the Laplacian operator

$$ \nabla^2 C,$$
where $C$ is some scalar or vectorial field. 

The Laplacian operator is widely used in various fields such as physics, mathematics and engineering. Some equations involving this operator are:
1. **Poisson's Equation**:
      $$ \nabla^2 \phi = \rho $$
2. **Heat Equation**:
 $$ \frac{\partial T}{\partial t} = \alpha \nabla^2 T, $$

3. **Wave Equation**:
  $$ \frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u, $$

4. **Schrödinger Equation**:
 $$ i\hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m} \nabla^2 \psi + V\psi, $$

5. **Laplace's Equation**:
  $$ \nabla^2 \phi = 0 ,$$
 
6. **Helmoltz equation**:
$$ \nabla (D \nabla  A) - \lambda A = B.$$
   
The objective here is to build a solver for a reaction-diffusion equation. The work is divided as follows : (i) study of the diffusion equation in 1D, (ii) study of the 2D Laplace equation 2D using iterative methods and (iii) study of the 2D Poisson-Helmholtz equation, for which we will write a solver directly useful for solving the reaction-diffusion equation.

### Table of contents


1. [Diffusion Equation in 1D](#diffusion-equation-in-1d)
    1. [Analytical solution](#analytical-solution)
    2. [Finite Differences Scheme : explicit method](#explicit_method)    
    3. [Finite Differences Scheme : implicit method](#implicit_method)
    4. [Compare Both Approaches](#compare-both-approaches)
2. [Laplace Equation in 2D](#laplace-equation-in-2d)
    1. [Jacobi Approach](#jacobi-approach)    
    2. [Gauss-Seidel](#gauss-seidel)
3. [Poisson-Helmholtz Equation](#poisson-helmholtz-equation)
4. [Reaction-Diffusion Equation](#reaction-diffusion-equation)

## 1 Diffusion equation in 1D
 
The equation describing the 1D diffusion of a chemical contaminant of concentration $C$ and diffusion coefficient $D$ is written 
   $$ \frac{\partial C}{\partial t} = D \frac{\partial^2 C}{\partial x^2} $$
    
The fluid domain is bounded by two absorbing walls located at $x=0$ and $x=L$, leading to the following boundary conditions : 

$$ C(0, t) = C(L, t)= 0 $$

Furthermore, we assume the following initial condition: 
$$ C(x, 0) = \sin(\frac{\pi x}{L})$$
 


### 1.1 Analytical solution

Using the Python library sympy verify that
$$ C = \exp\left(-\frac{D\pi^2 t}{L^2}\right)\sin\left(\frac{\pi x}{L}\right)$$
is a solution and represent it on a plot at different times. 


In [1]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sps
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

### 1.2 Finite Difference scheme : explicit method

In the finite differences scheme, we discretize the spatial and temporal domains to approximate the derivatives in the heat equation.
The spatial domain is divided into $N_x$ grid points with spacing $dx$, and the time domain is divided into $N_t$ time steps with spacing $ dt $.



Using finite differences, the second-order spatial derivative can be approximated as:

$$ \frac{\partial^2 C}{\partial x^2} \approx \frac{C_{i-1} - 2C_i + C_{i+1}}{\Delta_x ^2} $$

The forward Euler method is used for the time derivative:
   
$$ \frac{\partial C}{\partial t} \approx \frac{C^{n+1} - C^n}{\Delta t} $$

Combining these approximations, the concentration at time $t_{n+1}$ is expressed as a function of the concentration field at time $t_n$ (explicit method) as:

$$ C_i^{n+1} = C_i^n + \frac{D  \Delta t}{\Delta_x ^2} \left( C_{i-1}^n - 2C_i^n + C_{i+1}^n  \right) $$
where $C_i^n$ is the concentration at spatial position $x_i$ and time $ t_n $
 

Q1: Write a function ```rhs_centered(C,dx, D)``` where $C$ is the concentration array at the current-time step, that returns the right-hand side of the 1D diffusion equation based on centered finite differences. Be  careful to impose the correct boundary conditions. 

Q2: Write a function ```euler_step(C, f, dt, dx, D)``` that returns the concentration field $C(x, t+dt)$ given the concentration field $C(x, t)$, where $f(C, dx, D)$ is the right-hand side of the 1D diffusion equation based on centered finite differences. 

Q3: Write a function ```exact_solution(x,t,D, L)```returning the exact analytical solution. 

Q4: Solve the 1D diffusion equation using the explicit finite difference scheme. Compare with the analytical solution. 

### 1.3 Finite difference scheme : implicit method

This time, we use a backward Euler method for the time derivative. Combining with the centered finite difference approximation of the right-hand side of the 1D diffusion equation: 
 
$$ C_i^{n+1} - \frac{D \Delta t}{\Delta x^2} \left( C_{i-1}^{n+1} - 2C_i^{n+1} + C_{i+1}^{n+1} \right) = C_i^n $$

This can be rearranged into a system of linear equations:
$$ \left( 1 + 2\frac{D \Delta t}{\Delta x^2} \right) C_i^{n+1} - \frac{D \Delta t}{\Delta x^2} \left( C_{i-1}^{n+1} + C_{i+1}^{n+1} \right) = C_i^n $$

It can be put into the matrix form $A x = b$, where $x=C^{n+1}$ and $b=C^n$.

Q1: Write a function ```matrix_implicit(D, dt, dx)```than returns the sparse matrix A. 

Q2: Solve the 1D diffusion equation using the implicit finite difference scheme (use ```spsolve```) and compare with the analytical solution. 

### 1.4 Compare both approaches

Compare the explicit and implicit methods and perform a stability analysis (by varying the time step $dt$).  

## 2 Laplace equation in 2D

Inverting matrices for solving the diffusion equation in 2D can be complicated, we propose then to study some iterative approaches.

We study first the 2D Laplace equation 
$$ \nabla^2 C = \frac{\partial^2 C}{\partial x^2} + \frac{\partial^2 C}{\partial y^2}=0 $$

and we consider the fluid domain $[0, 1] \times [0, 1]$. The boundary conditions are: 
$$ C(x=0, y)=y, \quad C(x=1, y)=y-1, \quad C(x, y=0)=-y, \quad C(x, y=1)=1-y$$. 

The 5-point Laplacian is a finite difference approximation of the Laplace operator in two dimensions. It is commonly used in numerical methods to solve partial differential equations such as the Poisson equation. 


Using the 5-point stencil, the discrete approximation of the Laplacian at a grid point $(i, j)$ is

$$ \nabla^2 C_{i,j} \approx \frac{C_{i+1,j} - 2C_{i,j} + C_{i-1,j}}{\Delta x^2} + \frac{C_{i,j+1} - 2C_{i,j} + C_{i,j-1}}{\Delta y^2} $$
   
For a uniform grid where $\Delta x = \Delta y = h$, this simplifies to:
  
$$ \nabla^2 C_{i,j} \approx \frac{C_{i+1,j} + C_{i-1,j} + C_{i,j+1} + C_{i,j-1} - 4C_{i,j}}{h^2} $$

This formula is used to approximate the second derivatives in the Laplace equation, providing a way to discretize the equation for numerical solutions.
The iterative process is
$$ C_{i,j} = \frac{1}{4} ( C_{i+1,j} + C_{i-1,j} + C_{i,j+1} + C_{i,j-1} ) $$
 

### 2.1 Jacobi approach

Q1: First define a function ```jacobi(grid)``` that returns a new concentration field from an initial field ```grid```using the iterative process: 
$$ C_{i,j} = \frac{1}{4} ( C_{i+1,j} + C_{i-1,j} + C_{i,j+1} + C_{i,j-1} ) $$
Don't forget to copy the boundary conditions of the initial field. 


Q2: Define a function ```boundary(grid)```that imposes the required boundary conditions on a given ```grid```. 

Q3: Define a function ```initgrid(gridsize)``` that randomly initializes a grid of size ```gridsize```x ```gridsize```. You may use the function ```np.random.randn```. Don't forget to impose the required boundary conditions. 


Q4: Solve the 2D Laplace equation on a grid of size $25\times 25$ and represent the solution at different iteration stops.  To represent the solution, you may run first the following cell and use the function ```showsol(grid)```. 

In [2]:
import matplotlib as ml
def showsol(sol):
    plt.imshow(sol.T,cmap=ml.cm.Blues,interpolation='none',origin='lower')
    cbar=plt.colorbar()
    cbar.set_label('C(x,y)')

Q5: Perform a convergence analysis by calculating the error at each iteration, computed as the difference between the norm of the final solution (obtained after many iterations) and the one of the current grid.  

### 2.2 Gauss-Seidel approach 

We now turn to the Gauss-Seidel method. Starting with an initial guess for $C$ it updates the solution at each grid point using the formula above. The key idea of Gauss-Seidel is that each time a grid point $C(i, j)$ is updated, the new value is immediately used in subsequent updates within the same iteration. This makes the method faster to converge compared to the Jacobi method, which updates all points after a full sweep of the grid.

Q1: Define a function ```gauss-seidel(grid)```that returns the grid updated with a Gauss-Seidel method. 

Q2:  Perform a convergence analysis and compare with the Jacobi method. 

## 3 The Poisson-Helmholtz equation 

The Poisson-Helmholtz equation is a partial differential equation that combines aspects of both the Poisson and Helmholtz equations. It is commonly used in various fields such as physics and engineering to describe phenomena like wave propagation, electrostatics, and heat conduction.

The general form of the Poisson-Helmholtz equation is:
$$ \nabla (D \nabla  A) - \lambda A = B$$
where $A$ is a scalar field, $D$ is the diffusion coefficient, $\lambda$ is a constant and $B$ is the source term.  

In the context of numerical methods, solving the Poisson-Helmholtz equation typically involves discretizing the equation using finite difference or finite element methods and then solving the resulting system of linear equations.
  
We aim at solving the Poisson-Helmoltz equation in 2D. To do so, we will assume a constant diffusion coefficient $D$. Under this assumption, the equation can be discretized as follow: 

$$ D  (A_{i+1,j} + A_{i-1,j} + A_{i,j+1} + A_{i,j-1} - 4 A_{i,j}) - \lambda  A_{i,j} =  B_{i,j}$$

The iterative algorithm is then:
 $$ A_{i,j} = {- B_{i,j} + D  (A_{i+1,j} + A_{i-1,j} + A_{i,j+1} + A_{i,j-1}) \over 4 D + \lambda}$$
 
  

Q1: Define a function ```poisson_helmoltz_jacobi(u,D,l,b)```that, given a grid $u$, returns the updated grid by implementing the above iterative algorithm, with the Jacobi method. 

Q2: Solve the equation for $\lambda=0$ and $B=0$ and the same initial and boundary conditions as used to solve the 2D Laplace equation. Make sure you obtain the same results than previously, by plotting the solution after various iteration numbers and comparing with the plots obtained solving the Laplace equation. 

Q3: Define a function ```poisson_helmoltz_gs(u,D,l,b)```that, given a grid $u$, returns the updated grid by implementing the above iterative algorithm, with the Gauss-Seidel method. 

Q4: Perform a convergence analysis and compare both methods in the case of the 2D Poisson-Helmholtz equation. You may use $\lambda=0.1$ and $B$ corresponding to a point source located at coordinates $(0.5, 0.5)$. 

## 4 Reaction-Diffusion equation

The general reaction-diffusion equation reads:
$$ \frac{\partial C}{\partial t} = D \nabla^2 C + \beta C + R, $$
where $R$ is a source term and $\beta$ is a growth rate of the substance of concentration $C$. 

This equation can be discretized into :
$$ \frac{C^{n+1} - C^n}{\Delta t} =  D \nabla^2 C^{n+1} + \beta C^{n+1} + R$$ 

We can write it in a Poisson-Helmoltz form: 
$$ D \nabla^2 C^{n+1} + (\beta  - \frac{1}{dt}) C^{n+1} = - \frac{1}{dt} C^n - R $$
Thus, noting $\lambda = \beta  - \frac{1}{dt}$ and $B =  -\frac{1}{dt} C^n - R $, the discrtized reaction diffusion equation writes: 
 
  $$ D \nabla^2 C^{n+1} + \lambda C^{n+1} = B$$
  
Thus, you have now an algorithm to solve reaction-diffusion problems in 2D ! 
 